# Q-Learning For House Navigation Task



In [1]:
# Here we will use numpy arrays
import numpy as np


## Define State and Action spaces

Here we use discrete sets of states and actions.


In [2]:
S = ['A','B','C','D','E','F']
A = ['A','B','C','D','E','F']

## Define R Matrix

R matrix indicates the actions which are possible from any given state, along with the numeric reward for every possible action in each state.

The R matrix is part of the environment. It is constructed by the task designer, and the agent is unable to change it in any way. 

We use **np.NaN** (i.e. "not a number") to indicate actions which are impossible.

Note: Numpy indexes by 0, so the indices in the R matrix for this example go from 0 to 5 instead of 1 to 6.

In [3]:
# Initialize R matrix with np.nan in all cells
R =  np.array([[np.nan for a in A] for s in S])

# Populate R matrix with possible actions + rewards until it matches the R matrix given in the tutorial notes.
R[0, 4] = 0 # door from room A to E, reward=0
R[1, 3] = 0 # door from B to D, reward=0
R[1, 5] = 100 # door from B to D, reward=100

# Slightly faster: define these as a list.
possible_actions = [(0,4), (1,3), (1,5), (2,3), (3, 1), (3,2), (3,4), (4,0), (4,3), (4,5), (5,1), (5,4), (5,5)]

for s,a in possible_actions:
    if a == 5:
        R[s,a] = 100
    else:
        R[s,a] = 0
        
print('R matrix: \n\n{}'.format(R))

R matrix: 

[[ nan  nan  nan  nan   0.  nan]
 [ nan  nan  nan   0.  nan 100.]
 [ nan  nan  nan   0.  nan  nan]
 [ nan   0.   0.  nan   0.  nan]
 [  0.  nan  nan   0.  nan 100.]
 [ nan   0.  nan  nan   0. 100.]]


## Define Q Matrix

The Q matrix indicates the agent's valuation for the given state and action pair. In other words, the agent's expected return for taking the given action in the given state. 

When the agent knows the number of possible states and actions already, the Q matrix is the same size as the R matrix. The indices are the same, ie row 0 indicates state A etc in the Q matrix as well as the R matrix.

We initialize the Q matrix with zeros. Right now, the agent doesn't know anything about the value of any given action in any given state, and it also doesn't know which actions are possible in which state yet.

In [4]:
Q = np.zeros(R.shape)

print('Q matrix: \n\n{}'.format(Q))

Q matrix: 

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


## Begin!

Okay now we're ready for our agent to start learning. We will do the first timestep step-by-step to see how it works, and then see how it all comes together by looping over timesteps within a single episode.

### Initialize State
Here we randomly initialize the state (ie randomly select which room the agent starts in) to begin the episode.

In [5]:
s = np.random.choice(len(S))
print("Starting state is '{}', which is row {} in the Q and R matrices".format(S[s], s))

Starting state is 'C', which is row 2 in the Q and R matrices


### Identify Available Actions for State
Now we identify which are our available actions in this state from the R matrix.

Note: the available actions are the elements which are NOT nan from the row of the R matrix corresponding to current state.

In [6]:
available_actions = np.where(~np.isnan(R[s]))[0]

a_idx = list(available_actions)
a_labels= [A[x] for x in available_actions]
q_values = [Q[s,a] for a in a_idx]

print('Available actions are: {}'.format(a_labels))
print('Those actions correspond to column indices: {}'.format(a_idx))
print('Those actions have estimated values: {}'.format(q_values))

Available actions are: ['D']
Those actions correspond to column indices: [3]
Those actions have estimated values: [0.0]


### Select Action

Here we are using a **greedy behavioural policy**, so we always choose the action with the highest value. Ties are broken at random.

Later in the notebook we switch to epsilon-greedy, which chooses a greedy action with probability 0.9 and a random action with probability 0.1.

In [7]:
best_actions = available_actions[np.where(q_values == np.max(q_values))[0]]
best_actions_labels = [A[x] for x in best_actions]
best_actions_q_values = [Q[s,a] for a in best_actions]

print("Our best available actions from state '{}' are: {} with current q values: {}".format(S[s],
    best_actions_labels, best_actions_q_values))

a = np.random.choice(best_actions)

print("Randomly selecting action '{}' with current Q value {}".format(A[a], Q[s,a]))

Our best available actions from state 'C' are: ['D'] with current q values: [0.0]
Randomly selecting action 'D' with current Q value 0.0


## Update Environment (According to Action Selected by Agent)

### Determine reward

The reward is issued by the environment given the state and action pair

In [8]:
r = R[s,a]
print("Reward for taking action '{}' from state '{}': {}".format(A[a], S[s], r))

Reward for taking action 'D' from state 'C': 0.0


### Update State

The environment state updates given the state action pair

In [9]:
s_old = s
s = a # here, the transition function is deterministic. Next state corresponds simply to the action taken.
print("After taking action '{}' from state '{}', new state is '{}'".format(A[a], S[s_old], S[s]))

After taking action 'D' from state 'C', new state is 'D'


## Update Q Matrix

Now that the agent has observed some reward, it can update its internal value estimates.

Note that if the reward received was zero, the update is also zero! We don't learn anything until we receive a non-zero reward.

In [10]:
alpha = 1
gamma = 0.8

Q[s_old,a] = Q[s_old,a] + alpha * ( r + gamma * np.max(Q[s,:]) - Q[s_old,a])

print('Q matrix updated: \n\n {}'.format(Q))

Q matrix updated: 

 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


# Run Through a Whole Episode

Now that we've seen a single update step-by-step, let's put it all together and run through a whole episode.

## Reset Learning

Run the cell below to initialize the matrices to their original values and reset learning.


In [11]:
# Learning params
alpha = 1
gamma = 0.8
epsilon=0.9

# States and Actions
S = ['A','B','C','D','E','F']
A = ['A','B','C','D','E','F']

goal_state = 'F'

# R matrix
R =  np.array([[np.nan for a in A] for s in S])
possible_actions = [(0,4), (1,3), (1,5), (2,3), (3, 1), (3,2), (3,4), (4,0), (4,3), (4,5), (5,1), (5,4), (5,5)]

for s,a in possible_actions:
    if a == 5:
        R[s,a] = 100
    else:
        R[s,a] = 0

# Q matrix
Q = np.zeros(R.shape)

## Run An Episode

Run the cell below to run a single episode. 

All information is printed in the cell output.

How many episodes do you have to run to get to the final Q values?

In [12]:
s = np.random.choice(len(S))
print("Starting state is '{}'".format(S[s]))

for i in range(500):
    # Action selection
    available_actions = np.where(~np.isnan(R[s]))[0]
    print("Available actions from state '{}' are: {}".format(S[s], [A[x] for x in available_actions]))

    q_values = [Q[s,a] for a in available_actions]
    print('Q values for those actions from current state: {}'.format(q_values))

    best_actions = available_actions[np.where(q_values == np.max(q_values))[0]]
    best_actions_q_values = [Q[s,x] for x in best_actions]

    if len(best_actions) > 1:
        print('Detected multiple actions with identical Q values. Agent will randomly select one of these.')
        print('Our best available actions from here are: {} with current q values: {}'.format(
            [A[x] for x in best_actions], best_actions_q_values))
    
    # Epsilon-greedy
    if np.random.uniform() > epsilon:
        a = np.random.choice(available_actions)
        print("Selecting random action '{}' with current Q value {}".format(A[a], Q[s,a]))
    else:
        a = np.random.choice(best_actions)
        print("Selecting greedy action '{}' with current Q value {}".format(A[a], Q[s,a]))


    # Environment updating
    r = R[s,a]
    print("Reward for taking action '{}' from state '{}': {}".format(A[a], S[s], r))

    s_old = s
    s = a # here, the transition function is deterministic. Next state corresponds simply to the action taken.
    print("After taking action '{}' from state '{}', new state is '{}'".format(A[a], S[s_old], S[s]))

    # Q value updating
    q_updated = Q[s_old,a] + alpha * ( r + gamma * np.max(Q[s,:]) - Q[s_old,a])
    Q[s_old,a] = q_updated

    print("Q value update: " \
          "Q({},{}) = Q({},{}) + alpha*(r({},{}) + gamma*max(Q[{},:]) - Q[{},{}]) ".format(
        S[s_old], A[a], S[s_old], A[a], S[s_old], A[a], S[s],S[s_old], A[a]))

    print("Q matrix update: " \
          "Q({},{}) = {} + {}*({} + {}*{} - {}) = {}".format(
        S[s_old], A[a], Q[s_old,a].round(0), alpha, r, gamma, np.max(Q[s,:]).round(0), 
        Q[s_old,a].round(0), q_updated))

    print('Q matrix updated: \n\n {}'.format(Q.round(0)))

    if S[s] == goal_state:
        print("Goal state '{}' reached. Ending episode.".format(goal_state))
        break


Starting state is 'E'
Available actions from state 'E' are: ['A', 'D', 'F']
Q values for those actions from current state: [0.0, 0.0, 0.0]
Detected multiple actions with identical Q values. Agent will randomly select one of these.
Our best available actions from here are: ['A', 'D', 'F'] with current q values: [0.0, 0.0, 0.0]
Selecting greedy action 'F' with current Q value 0.0
Reward for taking action 'F' from state 'E': 100.0
After taking action 'F' from state 'E', new state is 'F'
Q value update: Q(E,F) = Q(E,F) + alpha*(r(E,F) + gamma*max(Q[F,:]) - Q[E,F]) 
Q matrix update: Q(E,F) = 100.0 + 1*(100.0 + 0.8*0.0 - 100.0) = 100.0
Q matrix updated: 

 [[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.]
 [  0.   0.   0.   0.   0.   0.]]
Goal state 'F' reached. Ending episode.


## Run Until Convergence

Run many episodes until Q matrix converges to final values

In [17]:
# Learning params
alpha = 1
gamma = 0.8
epsilon = 0.9

# States and Actions
S = ['A','B','C','D','E','F']
A = ['A','B','C','D','E','F']

goal_state = 'F'

# R matrix
R =  np.array([[np.nan for a in A] for s in S])
possible_actions = [(0,4), (1,3), (1,5), (2,3), (3, 1), (3,2), (3,4), (4,0), (4,3), (4,5), (5,1), (5,4), (5,5)]

for s,a in possible_actions:
    if a == 5:
        R[s,a] = 100
    else:
        R[s,a] = 0

# Q matrix
Q = np.zeros(R.shape)

# Run
for episode in range(1000):
    
    s = np.random.choice(len(S))
    print("Starting state is '{}'".format(S[s]))
    
    for timestep in range(500):
        # Action selection
        available_actions = np.where(~np.isnan(R[s]))[0]
        q_values = [Q[s,a] for a in available_actions]
        best_actions = available_actions[np.where(q_values == np.max(q_values))[0]]
        best_actions_q_values = [Q[s,x] for x in best_actions]
        
        # Epsilon-greedy
        if np.random.uniform() > epsilon:
            a = np.random.choice(available_actions)
        else:
            a = np.random.choice(best_actions)

        # Environment updating
        r = R[s,a]
        s_old = s
        s = a 

        # Q value updating
        Q[s_old,a] = Q[s_old,a] + alpha * ( r + gamma * np.max(Q[s,:]) - Q[s_old,a])

        if S[s] == goal_state:
            break
        
    print('Episode {} finished. Q matrix values:\n{}'.format(episode,Q.round(1)))

Starting state is 'D'
Episode 0 finished. Q matrix values:
[[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]
Starting state is 'B'
Episode 1 finished. Q matrix values:
[[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]
Starting state is 'A'
Episode 2 finished. Q matrix values:
[[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.]
 [  0.   0.   0.   0.   0.   0.]]
Starting state is 'D'
Episode 3 finished. Q matrix values:
[[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.  80.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.]
 [

In [16]:
print('Final Q matrix: \n{}'.format(Q.round(0)))

Final Q matrix: 
[[  0.   0.   0.   0. 400.   0.]
 [  0.   0.   0. 320.   0. 500.]
 [  0.   0.   0. 320.   0.   0.]
 [  0. 400. 256.   0. 400.   0.]
 [320.   0.   0. 320.   0. 500.]
 [  0. 400.   0.   0. 400. 500.]]


# Further exercises:

- Explore the impact of different parameters (e.g. learning rate, discount rate), experiment with different values
- Plot observations for different parameter values
- Perform a grid search to find the optimal combination
- In preparation for next week's lab, try to apply Q-Learning to your selected domain and repeat the above tasks